In [38]:
import gpflow
import tensorflow as tf
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
plt.style.use('ggplot')
%matplotlib inline

import os
os.environ["CUDA_VISIBLE_DEVICES"] = ''

import keras
from keras.datasets import mnist
from keras import backend as K
 

xs_train_full = np.genfromtxt("../data/mnist_train_features.csv", delimiter=",")
xs_test_full = np.genfromtxt("../data/mnist_test_features.csv", delimiter=",")

def get_mnist_classes():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    return ((x_train, x_test), (y_train, y_test))

((mnist_train_full, mnist_test_full), (ys_train_full, ys_test_full)) = get_mnist_classes()


In [4]:
# fitting entirety of model on base MNIST data

num_features = xs_train_full.shape[-1]
num_classes = 10
print("Using", num_features, "features, predicting", num_classes, "classes")

ys = np.argmax(ys_train_full, axis=1)

full_model = gpflow.models.SVGP(
    xs_train_full, ys, kern=gpflow.kernels.Matern32(input_dim=num_features) + gpflow.kernels.White(input_dim=num_features, variance=0.2),
    likelihood=gpflow.likelihoods.MultiClass(num_classes),
    Z=xs_train_full[::40].copy(), 
    num_latent=num_classes, 
    whiten=True, 
    q_diag=True,
    minibatch_size=5000)
full_model.kern.white.variance.trainable = False
full_model.feature.trainable = False

Using 128 features, predicting 10 classes


In [5]:
opt = gpflow.train.ScipyOptimizer()
opt.minimize(full_model)

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 6761.897608
  Number of iterations: 16
  Number of functions evaluations: 29


In [7]:
(mu, var) = full_model.predict_y(xs_test_full)
predicted_classes = np.argmax(mu, axis=1)
correct_classes = np.argmax(ys_test_full, axis=1)

In [10]:
# get the full CNN
mnist_cnn = keras.models.load_model('../models/mnist_cnn.h5')

In [25]:
img_rows, img_cols = 28, 28
if K.image_data_format() == 'channels_first':
    mnist_test_full = mnist_test_full.reshape(xs_test_full.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    mnist_test_full = mnist_test_full.reshape(xs_test_full.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    

mnist_test_full = mnist_test_full.astype('float32')
mnist_test_full /= 255

In [35]:
cnn_test_probs = mnist_cnn.predict_on_batch(mnist_test_full)
cnn_test_preds = np.argmax(cnn_test_probs, axis=1)

In [37]:
# Combined model

"""
params:
    cnn: trained mnist CNN
    gp: trained gaussian process for multiclass classification
    verbose: printing on/off 
    accept_cnn_stddev: if CNN predicted class has higher prob than GP's, this is number of stddevs the CNN prediction needs to be within from the GP prediction to acccept it
        ie. higher = more likely to revert to CNN!
"""
def combined_predict(cnn, gp, image, verbose=False, accept_cnn_stddev=1):
    # so my CNN can only predict on batches of 4 or greater...
    batch = np.array([image, image, image, image])
    cnn_probs = cnn.predict_on_batch(batch)[0]
    gp_mu, gp_var = gp.predict_y([image])
    gp_mu, gp_var = gp_mu[0], gp_var[0]
    
    cnn_class = np.argmax(cnn_probs)
    gp_class = np.argmax(gp_mu)
    
    # both classes agree
    if gp_class == cnn_class:
        # we may have to accept the wrong decision but can't do anything about it
        if verbose:
            print("Models agree on predicted class")
        return (0, gp_class, gp_mu, gp_var)
        
    else:
        # disagreement! This is additional information
        # From prior experiments we suspect that NN is more likely to be correct [non-adverserial examples tested]
        # So, if we take the CNN prediction and check if it's the same as the _second_ highest GP prediction
        # try using that?
        
        # Revised:
        #  Take the CNN prediction IF it's probability is within 1 stddev of the corresponding GP class probability
        
        # core idea: if CNN is _too_ sure then we revert to GP prediction -- might be adverserial...?
        
        gp_prob_for_cnn_pred = gp_mu[cnn_class]
        gp_stddev_for_cnn_pred = np.sqrt(gp_var[cnn_class])
        
        if verbose:
            print("Models disagree on predicted class")
            
        if cnn_pred_prob < (gp_prob_for_cnn_pred + accept_cnn_stddev*gp_stddev_for_cnn_pred):
            if verbose:
                print("  Taking CNN prediction, probability is within", accept_cnn_stddev, "stddev of GP probability")
            return (1, cnn_class, cnn_probs, [-1]*gp_var.shape[0])
            
        else:
            if verbose:
                print("  Taking GP prediction")
            return (0, gp_class, gp_mu, gp_var)
    


In [ ]:
# Have all 3 of MNIST CNN, trained GP, and combined model!


In [54]:
import h5py as h5


# Preprocess the N-MNIST data just like standard MNIST
nmnist_noisy = scipy.io.loadmat('../data/raw/n-mnist/mnist-with-awgn')
nmnist_blur = scipy.io.loadmat('../data/raw/n-mnist/mnist-with-motion-blur')
nmnist_noisy_lowcontrast = scipy.io.loadmat('../data/raw/n-mnist/mnist-with-reduced-contrast-and-awgn')

OSError: Unable to open file (file signature not found)